<a href="https://colab.research.google.com/github/HassanSherwani/Product_Purchase_Frequency/blob/master/Product_Freq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement:

How often a certain product has been sold in the past days.

#1)- Importing key modules

In [0]:
# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import sys

In [5]:
! pip install turicreate

In [0]:
import turicreate as tc

# 2)-Loading Data

In [0]:
url = 'https://raw.githubusercontent.com/HassanSherwani/Product_Purchase_Frequency/master/20190207_transactions%20.json'

In [0]:
transactions = pd.read_json(url, lines= True)

# 3)-Exploring dataset

In [11]:
transactions.head()

,id,products
0,0,"[185, 30, 77, 188, 78, 125, 45, 155, 241, 229,..."
1,1,"[119, 148, 108, 34, 157, 82, 113, 45, 165]"
2,2,"[173, 103, 229, 240]"
3,3,[91]
4,4,"[175, 192, 54, 172]"


In [12]:
transactions.shape

(2500, 2)

In [13]:
print(transactions['products'][1415])

[250, 236, 242, 229, 92, 2, 71, 172, 109, 247, 171, 209, 90, 139, 188, 191, 145, 214, 216, 237]


In [14]:
transactions.info() # checking missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 2 columns):
id          2500 non-null int64
products    2500 non-null object
dtypes: int64(1), object(1)
memory usage: 138.6+ KB


In [15]:
transactions.describe()

,id
count,2500.00000
mean,1249.50000
std,721.83216
min,0.00000
25%,624.75000
50%,1249.50000
75%,1874.25000
max,2499.00000


# 4)- Adding Features:

### 4.1)-Create a separate dataframe for recommending users

In [0]:
customers=transactions['id']

In [17]:
customers.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [0]:
import random
random.shuffle(customers) 

 To get random values of customer ids. This would help us in avoiding sample biaseness in our modeling.

In [0]:
cust_2_rec=customers[:1000]

In [20]:
cust_2_rec.head()

0     935
1    2201
2    2326
3     170
4    1472
Name: id, dtype: int64

### 4.2)- break down each list of items in the products column into rows and count the number of products bought by a user

In [21]:


pd.melt(transactions.head(2).set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,id,productId,purchase_count
0,935,30.0,1
1,935,45.0,1
2,935,77.0,1
3,935,78.0,1
4,935,89.0,2
5,935,125.0,1
6,935,133.0,1
7,935,155.0,1
8,935,161.0,1
9,935,185.0,1


# 5- Creating features for user_id,product_id and purchase count

### 5a)-One for purchase count

In [23]:
s=time.time()

data = pd.melt(transactions.set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.01 minutes


In [24]:
data.shape

(24811, 3)

In [25]:
data.head()

,id,productId,purchase_count
0,0,3,1
1,0,10,1
2,0,17,1
3,0,26,1
4,0,52,1


### 5-b)-Dummy as target

Dummy for marking an assumption whether a customer bought that item or not.
If one buys an item, then purchase_dummy are marked as 1.

In [0]:
 
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [0]:
data_dummy = create_data_dummy(data)

In [28]:
data_dummy.head()

,id,productId,purchase_count,purchase_dummy
0,0,3,1,1
1,0,10,1,1
2,0,17,1,1
3,0,26,1,1
4,0,52,1,1


### 5-c)-Normalize item

we normalize purchase frequency of each item across users by first creating a user-item matrix 

In [0]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')

In [30]:
df_matrix.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The NaN tells us that the item represented by the column was not purchased in that specific transaction.

In [31]:
df_matrix.shape

(2378, 250)

In [32]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

df_matrix_norm.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
print(df_matrix_norm.shape)

(2378, 250)


### 5-d)- create a table for input to the modeling

In [0]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()

In [35]:
data_norm.head()

,id,productId,scaled_purchase_freq
6,6,1,0.0
23,24,1,0.0
56,60,1,0.0
58,62,1,0.0
73,77,1,0.0


In [36]:
data_norm.shape

(22530, 3)

### 5-e) a function for normalizing data

In [0]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()


we have normalized the items accoreding to their purchase history, from 0–1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item.

# 6)-Split train and test set

we have three dataframes with purchase counts(data), purchase dummy(data_dummy), and scaled purchase counts(data_norm).

In [39]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(19848, 3) (4963, 3)


In [0]:
#convert dataframe to SFrame
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

A tabular, column-mutable dataframe object that can scale to big data <br>
https://turi.com/products/create/docs/generated/graphlab.SFrame.html

In [41]:
train_data

id,productId,purchase_count
18,28,1
1281,170,1
2182,51,1
1689,131,1
1561,246,1
2002,169,2
114,133,1
2416,79,1
2151,222,1
2126,66,1


In [42]:
test_data

id,productId,purchase_count
2189,140,1
1654,237,1
1847,118,1
456,174,1
1574,58,1
380,13,1
2207,237,1
368,231,1
66,202,2
1525,200,1


### 6.2)- Define a split_data function 

In [0]:
def split_data(data):

    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

### 6.3)-Apply for both dummy table and scaled/normalized purchase table

In [0]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

# 7)-Building Model

### 7.1)- Parameters to define field names for purchase count as target feature

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_count'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [48]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 19848 observations with 2346 users and 250 items.

Data prepared in: 0.041196s

19848 observations to process; with 250 unique items.

In [49]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 82555.9

+------+-----------+--------------------+------+
|  id  | productId |       score        | rank |
+------+-----------+--------------------+------+
| 935  |     96    | 1.0886075949367089 |  1   |
| 935  |    201    | 1.0694444444444444 |  2   |
| 935  |    153    | 1.0675675675675675 |  3   |
| 935  |    152    | 1.0673076923076923 |  4   |
| 935  |     44    | 1.0666666666666667 |  5   |
| 935  |    101    | 1.064102564102564  |  6   |
| 935  |    156    | 1.0632911392405062 |  7   |
| 935  |    122    | 1.063157894736842  |  8   |
| 935  |     85    | 1.063157894736842  |  9   |
| 935  |    207    | 1.0595238095238095 |  10  |
| 2201 |     96    | 1.0886075949367089 |  1   |
| 2201 |    125    | 1.069767441860465  |  2   |
| 2201 |    201    | 1.0694444444444444 |  3   |
| 2201 |    153    | 1.0675675675675675 |  4   |
| 2201 |    152    | 1.0673076923076923 |  5   |
| 2201 |     44    | 1.0666666666666667 |  6   |
| 2201 |    101    | 1.064102564102564  |  7   |
| 2201 |    156    |

### Note 

Through this model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations.<br> These 30 records include 3 users and their recommended items, along with score and descending ranks.

## Checking most frequent items(as per assignment) 

In [51]:
 train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
96     1.088608
125    1.069767
201    1.069444
153    1.067568
152    1.067308
44     1.066667
101    1.064103
156    1.063291
122    1.063158
85     1.063158
207    1.059524
50     1.058140
53     1.055556
186    1.054054
232    1.054054
88     1.053333
237    1.052632
129    1.052632
208    1.050000
181    1.050000
Name: purchase_count, dtype: float64

**products 96,125 201, 153, and 152 are the most popular (best-selling) across customers.**

### 7.2)- purchase dummy as target feature

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_dummy'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [55]:
popularity_model_dummy = tc.popularity_recommender.create(train_data_dummy, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 19848 observations with 2356 users and 250 items.

Data prepared in: 0.066983s

19848 observations to process; with 250 unique items.

In [56]:
popularity_recomm_4_dummy = popularity_model_dummy.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_dummy.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 65355.2

+------+-----------+-------+------+
|  id  | productId | score | rank |
+------+-----------+-------+------+
| 935  |     58    |  1.0  |  1   |
| 935  |     11    |  1.0  |  2   |
| 935  |     76    |  1.0  |  3   |
| 935  |    131    |  1.0  |  4   |
| 935  |     99    |  1.0  |  5   |
| 935  |    237    |  1.0  |  6   |
| 935  |    129    |  1.0  |  7   |
| 935  |    240    |  1.0  |  8   |
| 935  |     67    |  1.0  |  9   |
| 935  |    148    |  1.0  |  10  |
| 2201 |    199    |  1.0  |  1   |
| 2201 |     58    |  1.0  |  2   |
| 2201 |     11    |  1.0  |  3   |
| 2201 |     76    |  1.0  |  4   |
| 2201 |    131    |  1.0  |  5   |
| 2201 |     99    |  1.0  |  6   |
| 2201 |    237    |  1.0  |  7   |
| 2201 |    129    |  1.0  |  8   |
| 2201 |    240    |  1.0  |  9   |
| 2201 |     67    |  1.0  |  10  |
| 2326 |    199    |  1.0  |  1   |
| 2326 |     58    |  1.0  |  2   |
| 2326 |     11    |  1.0  |  3   |
| 2326 |     76    |  1.0  |  4   |
| 2326 |    131    |  1.0  |

### 7.3)- Applying 'scaled_purchase_freq' as target feature on model

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'scaled_purchase_freq'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [58]:
popularity_model_scaled = tc.popularity_recommender.create(train_data_norm, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 18024 observations with 2345 users and 227 items.

Data prepared in: 0.03905s

In [59]:
popularity_recomm_4_scaled = popularity_model_scaled.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_scaled.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 65655.6

+------+-----------+----------------------+------+
|  id  | productId |        score         | rank |
+------+-----------+----------------------+------+
| 935  |    201    | 0.08695652173913043  |  1   |
| 935  |     96    | 0.08333333333333333  |  2   |
| 935  |    129    | 0.07142857142857142  |  3   |
| 935  |    207    | 0.06976744186046512  |  4   |
| 935  |    156    | 0.06756756756756757  |  5   |
| 935  |     71    |        0.0625        |  6   |
| 935  |    186    | 0.058823529411764705 |  7   |
| 935  |    117    | 0.056338028169014086 |  8   |
| 935  |     26    | 0.05555555555555555  |  9   |
| 935  |     88    |  0.0547945205479452  |  10  |
| 2201 |    201    | 0.08695652173913043  |  1   |
| 2201 |     96    | 0.08333333333333333  |  2   |
| 2201 |    129    | 0.07142857142857142  |  3   |
| 2201 |    207    | 0.06976744186046512  |  4   |
| 2201 |    156    | 0.06756756756756757  |  5   |
| 2201 |     71    |        0.0625        |  6   |
| 2201 |    186    | 0.05882352

Great!.... We have all three models worked out. We have purchase frequency of each item customer-vice. <br>
Which one to trust and which one to discard? 

# 8)- Evaluate models

### 8.1)- Validation for Popularity Model on Purchase Counts

In [0]:
models_counts = [popularity_model]

In [0]:
model_names=['Popularity Model on Purchase Counts']

In [64]:
eval_counts = tc.recommender.util.compare_models(test_data, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/1921 queries. users per second: 54591.1


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.010411244143675193 | 0.004082075307999309 |
|   2    | 0.009370119729307656 | 0.006528717681762973 |
|   3    | 0.009023078257851817 |  0.0096737810168315  |
|   4    | 0.009890681936491431 | 0.015795344686547193 |
|   5    | 0.009370119729307652 | 0.018172578766019697 |
|   6    | 0.009456880097171626 | 0.022446146600233006 |
|   7    | 0.00944448575890532  | 0.02675813688307182  |
|   8    | 0.009435190005205617 | 0.02995959445725193  |
|   9    | 0.009890681936491421 | 0.03520487841154164  |
|   10   | 0.010150963040083293 | 0.03917685068127549  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.16440073512795547

Per User RMSE (best)
+------+------+-------+
|  id  | rmse | count |
+------+------+-------+

### 8.2)- Validation of Popularity Model on Purchase Counts (dummy)

In [65]:
models_counts = [popularity_model_dummy]
model_names=['Popularity Model on Dummy Purchase Counts']
eval_counts_dummy = tc.recommender.util.compare_models(test_data_dummy, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Dummy Purchase Counts


recommendations finished on 1000/1898 queries. users per second: 63319.2


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.010537407797681779 | 0.004075718801746212 |
|   2    | 0.011591148577449955 | 0.008843895830197188 |
|   3    | 0.010713031260976485 | 0.01199194640975462  |
|   4    | 0.010669125395152817 | 0.01667670229314066  |
|   5    | 0.009589041095890429 | 0.01863490390887649  |
|   6    | 0.009659290481208296 | 0.02249862010135983  |
|   7    | 0.009257865422248986 | 0.024970520347232704 |
|   8    | 0.009681243414120118 | 0.028927066084600408 |
|   9    | 0.009717831635639865 | 0.03285726830247377  |
|   10   | 0.010063224446786081 | 0.03746863866726877  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)
+-----+------+-------+
|  id | rmse | count |
+-----+------+-------+
| 871 | 0.0  |   1

### 8.3)- Validation for Popularity Model on Scaled Purchase Counts

In [66]:
models_counts = [popularity_model_scaled]
model_names=['Popularity Model on Scaled Purchase Counts']
eval_counts_dummy = tc.recommender.util.compare_models(test_data_norm, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/1834 queries. users per second: 51929.2


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.00926935659760088  | 0.003516902944383863 |
|   2    | 0.012268266085059976 | 0.01065976008724101  |
|   3    |  0.0123591421301345  | 0.016294074881861118 |
|   4    | 0.011859323882224649 | 0.01981162694085266  |
|   5    | 0.012213740458015276 | 0.025845796333800675 |
|   6    | 0.011904761904761897 | 0.02954120579529524  |
|   7    | 0.01230721296152048  | 0.03524822142597498  |
|   8    | 0.01254089422028354  | 0.04064625850340138  |
|   9    | 0.012480310190233835 | 0.04610487961087749  |
|   10   | 0.012268266085059985 | 0.05163923075591552  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.15419980587925075

Per User RMSE (best)
+-----+------+-------+
|  id | rmse | count |
+-----+------+-------+
| 

Dummy purchase count model gives lowest rmse . So we will choose that as best model for our final step.

# 9) Submission

In [68]:
users_to_recommend = list(cust_2_rec)

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy')


Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 24811 observations with 2378 users and 250 items.

Data prepared in: 0.046509s

...

| 25.284ms                            | 100              | 250             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.03294s

In [69]:
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 32512.9

+------+-----------+----------------------+------+
|  id  | productId |        score         | rank |
+------+-----------+----------------------+------+
| 935  |    168    | 0.025631868839263917 |  1   |
| 935  |     23    | 0.024235220750172932 |  2   |
| 935  |    248    | 0.023263971010843914 |  3   |
| 935  |    214    | 0.023167447249094645 |  4   |
| 935  |     63    | 0.020147351423899333 |  5   |
| 935  |     32    | 0.01888563632965088  |  6   |
| 935  |     74    | 0.01874724229176839  |  7   |
| 935  |    110    | 0.018568030993143716 |  8   |
| 935  |    191    | 0.018525727589925132 |  9   |
| 935  |     61    |  0.018173881371816   |  10  |
| 2201 |    112    | 0.02727676762474908  |  1   |
| 2201 |     17    | 0.027234587404463027 |  2   |
| 2201 |    214    | 0.025738835334777832 |  3   |
| 2201 |    149    | 0.025297747717963323 |  4   |
| 2201 |     80    | 0.02526219023598565  |  5   |
| 2201 |     2     | 0.02443495061662462  |  6   |
| 2201 |     23    | 0.02325500

### 9.2)- Checking most frequent items in final model

In [70]:
data_dummy.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
96     1.080808
207    1.076923
153    1.071429
201    1.069767
26     1.063158
152    1.060345
125    1.058824
112    1.058824
85     1.058824
131    1.057143
35     1.056180
63     1.054545
44     1.054545
129    1.053763
101    1.053763
117    1.052632
150    1.051724
122    1.051724
156    1.051546
120    1.050000
Name: purchase_count, dtype: float64

**products 96,207 , 153,201 and 26 are the most popular (best-selling) across customers.**

### 9.3)- CSV output file

In [72]:
df_rec = recom.to_dataframe()
df_rec.head()

,id,productId,score,rank
0,935,168,0.025632,1
1,935,23,0.024235,2
2,935,248,0.023264,3
3,935,214,0.023167,4
4,935,63,0.020147,5


In [73]:
print(df_rec.shape)


(10000, 4)


# 10)- Bonus Part- Recommending products to customers

In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates().sort_values('id').set_index('id')

In [76]:
recomendation = final_model.recommend(users=users_to_recommend, k=n_rec)

recommendations finished on 1000/1000 queries. users per second: 51615.6

In [0]:
df_rec = recomendation.to_dataframe()


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))

In [0]:

df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates() \
        .sort_values('id').set_index('id')

In [80]:
df_output.head()

,recommendedProducts
id,
1,161|2|85|160|17|143|206|59|54|152
5,45|228|7|1|144|100|29|145|195|74
11,175|249|130|214|97|45|152|159|2|72
19,133|130|127|100|56|52|26|17|10|3
21,228|13|212|166|147|17|224|63|161|2
